In [1]:
class Dqn_network():
    def __init__(self):
        self.lr = 0.01
        self.gamma = 0.9
        self.epsilon=1.0
        self.e_decay=0.9997
        self.e_min=0.01
        self.action_num=4
        self.memory_size = 20000
        #self.memory = Memory(self.memory_size)  -per
        self.memory = deque(maxlen=30000)
        self.batch_mini = 300
        self.model = self.build_model()
        self.target_model = self.build_model()
        # 타깃 모델 초기화
        self.update_target_model()
        
        
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    def remember_memory(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        
    def build_model(self):
        model = Sequential()
        model.add(Dense(64, input_shape=(1840,), activation='relu',kernel_initializer='he_normal'))
        #model.add(Flatten())
        #model.add(Dense(32, activation='tanh'))
        model.add(Dense(64,activation='relu',kernel_initializer='he_normal'))
        model.add(Dense(4, activation='sigmoid',kernel_initializer='he_normal'))
        model.compile(loss="mse", optimizer=Adam(lr=self.lr))
        model.summary()
        return model

    def dqn_input(self,state):
        state_list=np.zeros(1840)
        index=state[0]*460+state[1]
        state_list[index]=1
        state_list=state_list.reshape(1,1840)
        return state_list

    def choose_action(self,state):
        if self.epsilon >= random.random(): # or  (self.train_step_counter < self.observe)):
            return np.random.randint(0,self.action_num) 
        else:
            action_index=self.dqn_input(state)
            action_q = self.model.predict(action_index)
            action_index = rargmax(action_q[0])
            return action_index
        
    def replay_experience(self):
        batch_size=self.batch_mini
        minibatch = random.sample(self.memory, batch_size)
        actions, rewards, dones = [], [], []
        errors = np.zeros(batch_size)
        states = np.zeros((batch_size, 1840))
        next_states = np.zeros((batch_size, 1840))
        for i in range(batch_size):
            states[i] = self.dqn_input(minibatch[i][0])
            actions.append(minibatch[i][1])
            rewards.append(minibatch[i][2])
            next_states[i] = self.dqn_input(minibatch[i][3])
            dones.append(minibatch[i][4])

        target = self.model.predict(states)
        target_val = self.target_model.predict(next_states)
        # 벨만 최적 방정식을 이용한 업데이트 타깃
        for i in range(batch_size):
            if dones[i]:
                target[i][actions[i]] = rewards[i]
            else:
                target[i][actions[i]] = rewards[i] + self.gamma * (
                    np.amax(target_val[i]))
            
        self.model.fit(states, target, batch_size=batch_size,
                       epochs=1, verbose=0)
